# Target variables

TBD on lookup horizon - lookup over 1s candles instead of ticks to speed up


1. simple up/down signal
2. up/down/put where put would be either tails of distribution (moves less than 10th centile or greater than 90th)
3. actual up/down values in pips
4. up/down price movements transformed into 7 buckets of uniform distribution


# Features

Analytics across different candle resolutions such as 1min, 5min, 10min, 15min, 30min

gap value div prev 30days’ avg, median, 95th centile + time lag up to 5 periods
gap rank in the prev months gap + time lag up to 5 periods

last 10 candles how many down, how many up
last 5 candles size compared to: prev month mean, median, 95th centile

MA_{candleSize}_{period}_diff  + lag of 5
MA_{candleSize}_{period}_diff normalised by pervious month’s values + lag of 5

In [23]:
import featureeng as feng
import data.readers as rd
import time
import warnings
from warnings import simplefilter
from featureeng.features import *

def fill_data(df):
    df['close'] = df['close'].ffill()
    df['close'] = df['close'].bfill()
    
    df['midPoint'] = df['midPoint'].ffill()
    df['midPoint'] = df['midPoint'].bfill()
    
    df['Spread'] = df['Spread'].ffill()
    df['Spread'] = df['Spread'].bfill()
    return df

# Feature engineering

1. Load the data
2. For a given candle resolution
    1. Look within a defined time
    2. 

In [24]:
# 1. Load data for all years
# 2. b-fill f-fill for a given candle resolution (i.e. 10min)
# 3. gap cals
# 4. select sub-sample - 7:00 - 17:00

def prep_data(instrument='EURGBP', candle_size='10min', year=None):
    #1. Load data
    start_time = time.time()
    ticks = rd.read_in_ticks('/data', instrument, year=year)
    end_time = time.time()
    print(f'Loaded data in: {end_time - start_time:.3f} seconds')

    start_time = time.time()
    ticks =fill_data(process_df_count(ticks, candle_size))
    end_time = time.time()
    print(f'b-fill, f-fill in: {end_time - start_time:.3f} seconds')
    
    start_time = time.time()
    # Apply the custom function over a rolling window of size 3 with raw=True
    ticks['gap'] = ticks[['high', 'low', 'midPoint']].rolling(3, method='table').apply(f, raw=True, engine='numba')['high']
    
    # Print the DataFrame
    end_time = time.time()
    print(f'Gap calculation in: {end_time - start_time:.3f} seconds')
    return ticks.between_time('07:00', '19:00')


result_10min = prep_data()
result_1s = prep_data(candle_size='1s')

EURGBP - /data
/data/EURGBP
Loaded data in: 13.390 seconds
b-fill, f-fill in: 36.866 seconds
Gap calculation in: 0.038 seconds
EURGBP - /data
/data/EURGBP
Loaded data in: 13.449 seconds
b-fill, f-fill in: 50.220 seconds
Gap calculation in: 25.356 seconds


In [25]:
def prep_features(data):
    pass

1. simple up/down signal
2. up/down/put where put would be either tails of distribution (moves less than 10th centile or greater than 90th)
3. actual up/down values in pips
4. up/down price movements transformed into 7 buckets of uniform distribution

In [26]:
def prep_targets(data):
    pass


features_10min = prep_features(result_10min)
features_targets = prep_targets(result_1s)

In [27]:
result_1s.tail()

,open,high,low,close,midPoint,Count,Spread,gap
DateTime,,,,,,,,
2023-01-27 18:59:56,0.877030,0.877040,0.877030,0.877040,0.877035,5,0.40,-0.05
2023-01-27 18:59:57,0.877035,0.877045,0.877030,0.877045,0.877037,9,0.40,-0.10
2023-01-27 18:59:58,0.877040,0.877050,0.877040,0.877050,0.877045,5,0.36,0.00
2023-01-27 18:59:59,0.877040,0.877040,0.877035,0.877035,0.877038,2,0.45,-0.10
2023-01-27 19:00:00,0.877040,0.877045,0.877040,0.877045,0.877043,2,0.45,-0.05


In [28]:
#test

In [29]:
#def

In [30]:
#test